In [ ]:
# !wget https://datasets.aicrowd.com/default/aicrowd-practice-challenges/public/objde/v0.1/train.csv
# !wget https://datasets.aicrowd.com/default/aicrowd-practice-challenges/public/objde/v0.1/val.csv
# !wget https://datasets.aicrowd.com/default/aicrowd-practice-challenges/public/objde/v0.1/validation.zip
# !wget https://datasets.aicrowd.com/default/aicrowd-practice-challenges/public/objde/v0.2/train-v2.zip
# !wget https://datasets.aicrowd.com/default/aicrowd-practice-challenges/public/objde/v0.2/train-v2.csv

In [ ]:
# %cp /content/drive/MyDrive/Blitz_5/OBJDE/yolo_benchmark_70.zip /content/
!unzip /content/yolo_benchmark_70.zip

In [ ]:
!wget https://datasets.aicrowd.com/default/aicrowd-practice-challenges/public/objde/v0.2/test-v2.zip

In [ ]:
!unzip test-v2.zip

In [ ]:
%cp /content/train-v2.zip /content/drive/MyDrive/Blitz_5/OBJDE/
%cp /content/train-v2.csv /content/drive/MyDrive/Blitz_5/OBJDE/

# !unzip train.zip

In [ ]:
%cp /content/drive/MyDrive/Blitz_5/OBJDE/train-v2.csv /content/
%cp /content/drive/MyDrive/Blitz_5/OBJDE/train-v2.zip /content/


In [ ]:
!unzip train-v2.zip

In [ ]:
!git clone https://github.com/ultralytics/yolov5
!pip install thop
!pip install PyYAML==5.4.1

In [ ]:
!unzip ../validation.zip

unzip:  cannot find or open ../validation.zip, ../validation.zip.zip or ../validation.zip.ZIP.


In [ ]:
import pandas as pd
import numpy as np
import ast
import os
from os import path
import shutil
from tqdm import tqdm

In [ ]:
# train = pd.read_csv('train.csv')
val = pd.read_csv('train-v2.csv')
val.head(10)

,ImageID,LabelName,XMin,XMax,YMin,YMax
0,0000048549557964,Car,0.194375,0.326250,0.720000,0.758333
1,0000048549557964,Car,0.435000,0.725625,0.698333,0.944167
2,0000048549557964,Car,0.706250,0.936875,0.685833,0.865000
3,0000048549557964,Car,0.828750,0.999375,0.692500,0.908333
4,0000071d71a0a6f6,Person,0.000000,0.212500,0.231667,0.998333
5,0000071d71a0a6f6,Person,0.138750,0.522500,0.250000,0.998333
6,0000071d71a0a6f6,Person,0.142500,0.235000,0.418333,0.548333
7,0000071d71a0a6f6,Person,0.596250,0.808750,0.265000,0.998333
8,0000071d71a0a6f6,Person,0.638750,0.998750,0.243333,0.998333
9,0000201cd362f303,Clothing,0.020625,0.115000,0.382739,0.491557


In [ ]:
images= os.listdir('train-v2')

In [ ]:
train['LabelName'].value_counts()

In [ ]:
Clothing should be undersampled heavily

In [ ]:
val['LabelName'].value_counts()

Clothing    38935
Person      26365
Footwear    21389
Car          7469
Plant        7176
Name: LabelName, dtype: int64

##Preprocessing for YOLO


In [ ]:
key = dict({'Clothing':0, 'Person':1,'Footwear': 2, 'Plant': 3,'Car': 4})

In [ ]:
def tran(label):
  return str(key[label])

# train['class'] = train['LabelName'].apply(lambda a: tran(a))
val['class'] = val['LabelName'].apply(lambda a: tran(a))

In [ ]:
val['x_center'] = val['XMin'] + (val['XMax']-val['XMin'])/2
val['y_center'] = val['YMin'] + (val['YMax']-val['YMin'])/2
val['w'] = val['XMax'] - val['XMin']
val['h'] = val['YMax'] - val['YMin']

In [ ]:
def checktrain(value):
  if str(value)+'.jpg' in images[:5000]:
    return 1
  return 0
def checkval(value):
  if str(value)+'.jpg' in images[5000:5200]:
    return 2

In [ ]:
val['train'] = val['ImageID'].apply(checktrain)
val['train'] = val['ImageID'].apply(checkval)

In [ ]:
val['train'].value_counts()

In [ ]:
val

In [ ]:
val.head(10)

,ImageID,LabelName,XMin,XMax,YMin,YMax,class,x_center,y_center,w,h
0,0000048549557964,Car,0.194375,0.326250,0.720000,0.758333,4,0.260313,0.739167,0.131875,0.038333
1,0000048549557964,Car,0.435000,0.725625,0.698333,0.944167,4,0.580313,0.821250,0.290625,0.245834
2,0000048549557964,Car,0.706250,0.936875,0.685833,0.865000,4,0.821562,0.775416,0.230625,0.179167
3,0000048549557964,Car,0.828750,0.999375,0.692500,0.908333,4,0.914062,0.800416,0.170625,0.215833
4,0000071d71a0a6f6,Person,0.000000,0.212500,0.231667,0.998333,1,0.106250,0.615000,0.212500,0.766666
5,0000071d71a0a6f6,Person,0.138750,0.522500,0.250000,0.998333,1,0.330625,0.624167,0.383750,0.748333
6,0000071d71a0a6f6,Person,0.142500,0.235000,0.418333,0.548333,1,0.188750,0.483333,0.092500,0.130000
7,0000071d71a0a6f6,Person,0.596250,0.808750,0.265000,0.998333,1,0.702500,0.631667,0.212500,0.733333
8,0000071d71a0a6f6,Person,0.638750,0.998750,0.243333,0.998333,1,0.818750,0.620833,0.360000,0.755000
9,0000201cd362f303,Clothing,0.020625,0.115000,0.382739,0.491557,0,0.067812,0.437148,0.094375,0.108818


In [ ]:
val.shape

(101334, 11)

In [ ]:
val['class'].value_counts()

0    38935
1    26365
2    21389
4     7469
3     7176
Name: class, dtype: int64

In [ ]:
%cd /content/

/content


In [ ]:
# !rm -r /content/train/labels/*.txt
%cd /content/train/

/content/train


In [ ]:
!mkdir train

In [ ]:
 %cd /content/train
# !rm -r labels
!mkdir labels
!mkdir images

/content/train


In [ ]:
print(len(os.listdir('/content/val/labels')))

200


In [ ]:
base = "train/labels/"
for i in range(len(val)):
  if(path.exists(base+val.loc[i,"ImageID"]+'.txt')):
    with open(base+val.loc[i,"ImageID"]+".txt", "a") as file1: 
    # Writing data to a file 
      file1.write("\n") 
      file1.write("{} {} {} {} {}".format(val.loc[i,"class"],round(val.loc[i,"x_center"],6),round(val.loc[i,"y_center"],6),round(val.loc[i,"w"],6),round(val.loc[i,"h"],6))) 
      # print("{} {} {} {}".format(val.loc[i,"class"],round(val.loc[i,"x_center"],6),round(val.loc[i,"y_center"],6),round(val.loc[i,"w"],6),round(val.loc[i,"h"],6))) 
  else:
    with open(base+val.loc[i,"ImageID"]+".txt", "x") as file1: 
    # Writing data to a file  
      file1.write("{} {} {} {} {}".format(val.loc[i,"class"],round(val.loc[i,"x_center"],6),round(val.loc[i,"y_center"],6),round(val.loc[i,"w"],6),round(val.loc[i,"h"],6)))

In [ ]:
!unzip ../validation

In [ ]:
# !mv labels train
!mv ../validation train

mv: cannot stat '../validation': No such file or directory


In [ ]:

# !mkdir 
%cd val
!mkdir labels
!mkdir images
# images = [i for i in os.listdir('../train-v2')]
# images = [i for i in os.listdir('images')]


/content/val


In [ ]:
!rm -r /content/train/images/*.jpg

In [ ]:
len(images)

14800

In [ ]:
for i in images[:100]:
  shutil.move("/content/train/images/"+i, "/content/val/images/"+i)
  shutil.move("/content/train/labels/"+i.split('.')[0]+".txt", "/content/val/labels/"+i.split('.')[0]+".txt")

In [ ]:
for i in images[:10000]:
  shutil.move("/content/train-v2/"+i, "/content/train/images/"+i)
  # shutil.move("/content/train/labels/"+i.split('.')[0]+".txt", "/content/val/labels/"+i.split('.')[0]+".txt")

In [ ]:
for i in images[10000:10200]:
  shutil.move("/content/train-v2/"+i, "/content/val/images/"+i)
  shutil.move("/content/train/labels/"+i.split('.')[0]+".txt", "/content/val/labels/"+i.split('.')[0]+".txt")

In [ ]:
%cd yolov5/
!python train.py --batch 16 --data objde.yaml --cfg yolov5x.yaml --epochs 40 --weights /content/content/yolov5/runs/train/exp5/weights/last.pt --hyp /content/content/yolov5/runs/train/exp5/hyp.yaml 

In [ ]:
from google.colab import output
output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
 !python detect.py --source test-v2 --weights runs/train/exp9/weights/last.pt --save-txt --save-conf

In [ ]:
files = [i for i in os.listdir('/content/yolov5/runs/detect/exp2/labels')]
key = dict({0:'Clothing', 1:'Person',2:'Footwear', 3:'Plant',4:'Car'})

In [ ]:
df2 = pd.DataFrame(columns=['ImageID','LabelName','XMin','XMax','YMin','YMax','score']) 
for j in tqdm(files):
  with open("runs/detect/exp2/labels/"+j, "r") as file1:
    temp = [line[:-1] for line in file1]
    # print(temp)
    for i in temp:
      item = i.split(' ')
      classes = key[int(item[0])]
      x_min = float(item[1]) - float(item[3])/2
      x_max = float(item[1]) + float(item[3])/2
      y_min = float(item[2]) - float(item[4])/2
      y_max = float(item[2]) + float(item[4])/2
      score = float(item[5])
      df2.loc[len(df2.index)] = [ j.split('.')[0], classes, x_min, x_max, y_min, y_max, score]

100%|██████████| 9152/9152 [01:45<00:00, 86.56it/s]


In [ ]:
df2.to_csv('test1.csv')